In [11]:
import numpy as np
import pandas as pd

In [12]:
a = np.array([1,2,3,1,2,1,1,1,3,2,2,1,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10])
counts = np.bincount(a)
print(np.argmax(counts))

10


In [13]:
from collections import Counter 

def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 
    
List = ['a','b','a'] 
print(most_frequent(List)) 

a


In [46]:
df = pd.DataFrame()
df['prova']=[1,4,4,4,8]
df['test']=[100,300,300,300,100]
df['target']=[1,3,3,3,10]

In [44]:
df

,prova,test,target
0,1,1,1
1,2,3,3
2,2,3,3
3,4,3,3
4,5,10,10


In [27]:
def create_level_variable_matrix(dataframe,feature, local=None):
    
    x_j_k = np.unique(dataframe[feature].to_list())
    if local is None:
        x_most_freq = dataframe.apply(lambda x: most_frequent(x),axis=0)
    else:
        x_most_freq = dataframe.loc[local]
    
    Z = pd.DataFrame( x_most_freq ).T
    Z = pd.concat([Z]*len(x_j_k), ignore_index=True)
    Z[feature] = x_j_k
    Z['quantile'] = np.linspace(0,1,len(x_j_k))
    
    return Z

In [45]:
create_level_variable_matrix(df,'prova',local=0)

,prova,test,target,quantile
0,1,1,1,0.000000
1,2,1,1,0.333333
2,4,1,1,0.666667
3,5,1,1,1.000000


In [30]:
def create_quantile_feature_matrix(dataframe, feature, target, K, local = None):
    '''
    Create a matrix with K row, made by all the columns dataframe mean values, except for the "feature", which is replaced by K quantiles of the feature empiracal distribution
    '''
    if K > 2:
        quantile = np.linspace(0,1,K)
    if K == 2:
        quantile = [ 0.01, 0.99 ]
    x_j = dataframe[feature].values
    x_j_k = np.quantile(x_j,quantile)
    if local is None:
        x_mean = dataframe.drop(columns = [target]).mean()
    else:
        x_mean = dataframe.drop(columns = [target]).loc[local]
    Z = pd.DataFrame( x_mean ).T
    Z = pd.concat([Z]*len(x_j_k), ignore_index=True)
    Z[feature] = x_j_k
    Z['quantile'] = quantile
    
    return Z

In [33]:
a=create_quantile_feature_matrix(df,'prova','target',K=10)

In [39]:
b = a.apply(lambda x: most_frequent(x),axis=0)

In [42]:
pd.DataFrame(b).T

,prova,test,quantile
0,2.0,4.0,0.0


In [48]:
from statwolfml import Builder
model= 'random_forest_regressor'
build_model = Builder(model).feature_names([]).labels(['target']).build()
train_model = build_model.train(df)

In [54]:
print(train_model.base_model()['model'].predict(df[['test','prova']]))
print(train_model.base_model()['model'].predict(df[['prova','test']]))

[7.81 7.81 7.81 7.81 7.81]
[2.62 2.98 2.98 2.98 7.81]


In [56]:
train_model.base_model()['model'].predict(df.drop(columns='target'))

array([2.62, 2.98, 2.98, 2.98, 7.81])

In [61]:
def clean_list(List,to_keep):
    out = List.copy()
    for i in List:
        if i not in to_keep:
            out.remove(i)
    return out

In [63]:
clean_list(['a','b','c','d','e'],['b','a','e'])

['a', 'b', 'e']

In [64]:
from data_science.modules.load_dataset import create as createLD

In [70]:
createLD().classification_bin().dataframe()

,target,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708
